##### Import statements

In [2]:
from datetime import datetime
import pandas as pd
from utils import create_win_data, normalize, denormalize, features_and_labels, train_test_split, evaluate
from models import BiLSTM

In [3]:
df = pd.read_csv( "data/result/sp500-with-forbes-sentiment.csv", index_col="Date")

df.head()

,Open,High,Low,Close,Adj Close,Volume,Neg,Neu,Pos,Compound
Date,,,,,,,,,,
2018-01-02,2683.729980,2695.889893,2682.360107,2695.810059,2695.810059,3.397430e+09,0.12025,0.74875,0.13100,-0.042250
2018-01-03,2697.850098,2714.370117,2697.770020,2713.060059,2713.060059,3.544030e+09,0.00000,0.00000,0.00000,0.000000
2018-01-04,2719.310059,2729.290039,2719.070068,2723.989990,2723.989990,3.697340e+09,0.04325,0.78400,0.17275,0.431975
2018-01-05,2731.330078,2743.449951,2727.919922,2743.149902,2743.149902,3.239280e+09,0.00000,0.85100,0.14900,0.541300
2018-01-08,2742.669922,2748.510010,2737.600098,2747.709961,2747.709961,3.246160e+09,0.00000,0.94350,0.05650,0.318450


In [4]:
data = df[ [ "Open", "High", "Low", "Close", "Volume", "Compound", "Adj Close"]].values

print( f"Data Shape: { data.shape}")

Data Shape: (1559, 7)


##### Static variables

In [5]:
TRAIN_SPLIT = 0.85
SEQUENCE_LENGTH = 11
INPUT_DIM = data.shape[ -1]
INPUT_TIMESTEPS = 10
TRAIN_BATCH_SIZE = 8
TEST_BATCH_SIZE = 1
NEURONS = 50
STATEFUL = False
UNROLL = True
EPOCHS = 20

##### Split train and test data

In [6]:
train_data, test_data = train_test_split( data, TRAIN_SPLIT)

print( f"Train Data Shape: { train_data.shape}")
print( f"Test Data Shape: { test_data.shape}")

Train Data Shape: (1325, 7)
Test Data Shape: (234, 7)


##### Prepare train data

In [7]:
train_data_windows = create_win_data( train_data, SEQUENCE_LENGTH)

if STATEFUL:
	excess_windows = len( train_data_windows) % TRAIN_BATCH_SIZE
	train_data_windows = train_data_windows[ :len( train_data_windows) - excess_windows]

X_train, y_train = features_and_labels( train_data_windows)

normalized_train_data, record_min_train, record_max_train = normalize( train_data_windows)

X_train_normalized, y_train_normalized = features_and_labels( normalized_train_data)

print( f"Train Window Data shape: { train_data_windows.shape}")

Train Window Data shape: (1314, 11, 7)


##### Prepare test data

In [8]:
test_data_windows = create_win_data( test_data, SEQUENCE_LENGTH)

if STATEFUL:
	excess_windows = len( test_data_windows) % TEST_BATCH_SIZE
	test_data_windows = test_data_windows[ :len( test_data_windows) - excess_windows]

X_test, y_test = features_and_labels( test_data_windows)

normalized_test_data, record_min_test, record_max_test = normalize( test_data_windows)

X_test_normalized, y_test_normalized = features_and_labels( normalized_test_data)

print( f"Test Window Data shape: { test_data_windows.shape}")

Test Window Data shape: (223, 11, 7)


##### Bidirectional Long Short-term Memory (Bi-LSTM)

In [9]:
bi_lstm = BiLSTM(
	NEURONS,
	batch_size=TRAIN_BATCH_SIZE,
	window_size=INPUT_TIMESTEPS,
	input_dim=INPUT_DIM,
	stateful=STATEFUL,
	unroll=UNROLL
)

bi_lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 10, 100)           23200     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 100)               60400     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 83701 (326.96 KB)
Trainable params: 83701 (326.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


##### Train Perceptron

In [10]:
bi_lstm.fit(
	input=X_train_normalized,
	target=y_train_normalized,
	epochs=EPOCHS
)

##### Evaluate model prediction on train data

In [11]:
y_train_pred_normalized = bi_lstm.predict(
	input=X_train_normalized,
	batch_size=TRAIN_BATCH_SIZE
)

y_train_pred = denormalize( y_train_pred_normalized, record_min_train, record_max_train)

mse, rmse, mae, mape = evaluate( y_train, y_train_pred)
print( f"MSE: { mse:.2f}\nRMSE: { rmse:.2f}\nMAE: { mae:.2f}\nMAPE: { mape:.2f}%")

MSE: 1155.48
RMSE: 33.99
MAE: 24.41
MAPE: 0.71%


##### Evaluate model prediction on test data

In [12]:
y_test_pred_normalized = bi_lstm.predict(
	input=X_test_normalized,
	batch_size=TEST_BATCH_SIZE
)
y_test_pred = denormalize( y_test_pred_normalized, record_min_test, record_max_test)

mse, rmse, mae, mape = evaluate( y_test, y_test_pred)
print( f"MSE: { mse:.2f}\nRMSE: { rmse:.2f}\nMAE: { mae:.2f}\nMAPE: { mape:.2f}%")

MSE: 765.34
RMSE: 27.66
MAE: 22.63
MAPE: 0.50%


##### Further evaluation

In [ ]:
TRIES = 10

train_mse_list = []
train_rmse_list = []
train_mae_list = []
train_mape_list = []

test_mse_list = []
test_rmse_list = []
test_mae_list = []
test_mape_list = []

for _ in range( TRIES):
	bi_lstm = BiLSTM(
		neurons=NEURONS,
		batch_size=TRAIN_BATCH_SIZE,
		window_size=INPUT_TIMESTEPS,
		input_dim=INPUT_DIM,
		stateful=STATEFUL,
		unroll=UNROLL
	)

	bi_lstm.fit(
		input=X_train_normalized,
		target=y_train_normalized,
		epochs=EPOCHS
	)

	y_train_pred_normalized = bi_lstm.predict( input=X_train_normalized, batch_size=TRAIN_BATCH_SIZE)
	y_train_pred = denormalize( y_train_pred_normalized, record_min_train, record_max_train)

	train_mse, train_rmse, train_mae, train_mape = evaluate( y_train, y_train_pred)

	train_mse_list.append( train_mse)
	train_rmse_list.append( train_rmse)
	train_mae_list.append( train_mae)
	train_mape_list.append( train_mape)

	y_test_pred_normalized = bi_lstm.predict( input=X_test_normalized, batch_size=TEST_BATCH_SIZE)
	y_test_pred = denormalize( y_test_pred_normalized, record_min_test, record_max_test)

	test_mse, test_rmse, test_mae, test_mape = evaluate( y_test, y_test_pred)

	test_mse_list.append( test_mse)
	test_rmse_list.append( test_rmse)
	test_mae_list.append( test_mae)
	test_mape_list.append( test_mape)

now = datetime.strftime( datetime.now(), "%Y-%m-%dT%H-%M-%S")

pd.DataFrame( {
	"train_mse": train_mse_list,
	"train_rmse": train_rmse_list,
	"train_mae": train_mae_list,
	"train_mape": train_mape_list,
	"test_mse": test_mse_list,
	"test_rmse": test_rmse_list,
	"test_mae": test_mae_list,
	"test_mape": test_mape_list
}).to_csv( f"evaluations/sp500_BiLSTM_with_forbes_sentiment_{ now}.csv", index=False)